In [1]:
# Import necessary libraries
from flask import Flask, jsonify
import pandas as pd
import sqlite3

In [2]:
# Set up Flask app
app = Flask(__name__)

In [3]:
# Database path
sqlite_file = './Resources/hawaii.sqlite'

In [4]:
# Connect to the database
conn = sqlite3.connect(sqlite_file)
cursor = conn.cursor()

In [5]:
# Query all table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

print("Tables in the database:", tables)

Tables in the database: [('measurement',), ('station',)]


In [6]:
# Function to retrieve data from the database
def get_data():
    conn = sqlite3.connect(sqlite_file)

    # Precipitation data: Last 12 months
    latest_date_query = "SELECT MAX(date) FROM measurement;"
    latest_date = pd.read_sql(latest_date_query, conn).iloc[0, 0]
    one_year_ago = pd.to_datetime(latest_date) - pd.DateOffset(years=1)
    one_year_ago_str = one_year_ago.strftime('%Y-%m-%d')

    precipitation_query = f"""
    SELECT date, prcp
    FROM measurement
    WHERE date >= '{one_year_ago_str}';
    """
    precipitation_data = pd.read_sql(precipitation_query, conn)
    precipitation_dict = precipitation_data.set_index("date")["prcp"].to_dict()

    # Stations data
    stations_query = "SELECT station, name FROM station;"
    stations_data = pd.read_sql(stations_query, conn)
    stations_list = stations_data.to_dict(orient="records")

    # Most active station
    most_active_station_query = """
    SELECT station, COUNT(station) AS count
    FROM measurement
    GROUP BY station
    ORDER BY count DESC
    LIMIT 1;
    """
    most_active_station = pd.read_sql(most_active_station_query, conn).iloc[0, 0]

    # TOBS data: Last 12 months for the most active station
    tobs_query = f"""
    SELECT date, tobs
    FROM measurement
    WHERE station = '{most_active_station}' AND date >= '{one_year_ago_str}';
    """
    tobs_data = pd.read_sql(tobs_query, conn)
    tobs_list = tobs_data["tobs"].tolist()

    conn.close()

    return precipitation_dict, stations_list, tobs_list, most_active_station

In [7]:
# Load the data for the Flask app
precipitation_dict, stations_list, tobs_list, most_active_station = get_data()

In [8]:
# Flask Routes
@app.route("/")
def home():
    """List all available routes."""
    return (
        f"Welcome to the Climate API!<br/>"
        f"Available Routes:<br/>"
        f"/api/v1.0/precipitation<br/>"
        f"/api/v1.0/stations<br/>"
        f"/api/v1.0/tobs<br/>"
        f"/api/v1.0/&lt;start&gt;<br/>"
        f"/api/v1.0/&lt;start&gt;/&lt;end&gt;"
    )

@app.route("/api/v1.0/precipitation")
def precipitation():
    """Return JSON representation of precipitation data for the last 12 months."""
    return jsonify(precipitation_dict)

@app.route("/api/v1.0/stations")
def stations():
    """Return JSON list of stations."""
    return jsonify(stations_list)

@app.route("/api/v1.0/tobs")
def tobs():
    """Return JSON list of temperature observations for the most active station."""
    return jsonify(tobs_list)

@app.route("/api/v1.0/<start>")
@app.route("/api/v1.0/<start>/<end>")
def temperature_summary(start, end=None):
    """Return JSON list of TMIN, TAVG, and TMAX for a given date range."""
    conn = sqlite3.connect(sqlite_file)

    if end:
        query = f"""
        SELECT MIN(tobs) as TMIN, AVG(tobs) as TAVG, MAX(tobs) as TMAX
        FROM measurement
        WHERE date BETWEEN '{start}' AND '{end}';
        """
    else:
        query = f"""
        SELECT MIN(tobs) as TMIN, AVG(tobs) as TAVG, MAX(tobs) as TMAX
        FROM measurement
        WHERE date >= '{start}';
        """
    
    result = pd.read_sql(query, conn)
    conn.close()

    summary = {
        "TMIN": result["TMIN"].iloc[0],
        "TAVG": result["TAVG"].iloc[0],
        "TMAX": result["TMAX"].iloc[0]
    }
    return jsonify(summary)

In [9]:
# Run the Flask app in Jupyter
from werkzeug.serving import run_simple
import threading

def run_app():
    run_simple('localhost', 5000, app)

In [10]:
# Run the app in a separate thread
thread = threading.Thread(target=run_app)
thread.start()

 * Running on http://localhost:5000
Press CTRL+C to quit
